In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from BrainPulse import features_space,plot

In [2]:
#STFT
# df = features_space.load_features_csv_concat('/Users/luki/DataspellProjects/BrainPulse_Project/Results/stft')


In [3]:
e_list = ['Cz', 'CPz', 'POz', 'Pz', 'P1', 'P2', 'C3', 'C4', 'O1', 'O2', 'T7', 'T8', 'P3', 'P4', 'F3', 'F4']

In [32]:
#ERP STFT
df = features_space.load_features_csv_concat('/Users/luki/DataspellProjects/BrainPulse_Project/Results/epoch_stft')

In [ ]:
df

In [6]:
#ERP TDEMB
# df = features_space.load_features_csv_concat('/Users/luki/DataspellProjects/BrainPulse_Project/Results/erp_tdemb')

In [7]:
# import pandas as pd
# df = pd.concat([pd.read_csv('/Users/luki/DataspellProjects/BrainPulse_Project/Results/tdemb/1df_tdemb_4th_50.csv'),pd.read_csv('/Users/luki/DataspellProjects/BrainPulse_Project/Results/tdemb/2df_tdemb_4th_100.csv')]).reset_index(drop=True)

In [ ]:
temp_list = []

for sub_id in range(1,17):

    Subject_id = 'S'+ str(f'{sub_id:03}')

    df_Subject = df.groupby('Subject')
    S1 = df_Subject.get_group(Subject_id)
    Task = S1.groupby('Task')
    S1_A = Task.get_group('A')
    S1_AV = Task.get_group('AV')
    electrode_A = S1_A.groupby('Electrode')
    electrode_AV = S1_AV.groupby('Electrode')

    features_A = electrode_A['Lentr', 'TT', 'L', 'RR', 'LAM', 'DET', 'V','Vmax', 'Ventr', 'W', 'Wentr'].mean().apply(lambda x:x)
    names_cols_A = pd.DataFrame(np.array([[Subject_id]*16,['A']*16]).T,columns=['Subject', 'Task'])

    features_AV = electrode_AV['Lentr', 'TT', 'L', 'RR', 'LAM', 'DET', 'V','Vmax', 'Ventr', 'W', 'Wentr'].mean().apply(lambda x:x)
    names_cols_AV = pd.DataFrame(np.array([[Subject_id]*16,['AV']*16]).T,columns=['Subject', 'Task'])

    newA = pd.concat([names_cols_A,features_A.reset_index()],axis=1)
    newAV = pd.concat([names_cols_AV,features_AV.reset_index()],axis=1)
    subjectMean = pd.concat([newA,newAV]).reset_index(drop=True)
    temp_list.append(subjectMean)

new_df = pd.concat(temp_list).reset_index(drop=True)

In [ ]:
new_df


In [10]:
#TDEMB
# df = features_space.load_features_csv_concat('/Users/luki/DataspellProjects/BrainPulse_Project/Results/tdemb')


In [11]:
# df

In [37]:
df_e_wise, y = features_space.electrode_wise_dataframe(new_df,condition_list=['AV','A']) #['open_eyes','closed_eyes']

In [10]:
df_e_wise, y = features_space.electrode_wise_dataframe_epochs(df,condition_list=['AV','A']) #['open_eyes','closed_eyes']


In [ ]:
df_e_wise

In [39]:
df_norm = features_space.select_features_clean_and_normalize(df_e_wise,features=['Lentr', 'TT', 'L'])

In [ ]:
df_cv = features_space.cross_validation(df_norm,y,cv=10,type='linear')

In [ ]:
features_space.clasyfication_SVM(df_cv,y,cv=10,type='poly')

In [ ]:
features_space.clasyfication_RFC(df_cv,y,cv=10,max_depth=2)

In [ ]:
lin, lin_pred = features_space.compute_binary_SVM(df_cv,y,predict_on_all_data = True,type='linear')

In [ ]:
lin.coef_

In [ ]:
plot.SVM_histogram(df_cv, lin, lin_pred,'')


In [ ]:
plot.umap_on_condition(df_cv,y,'(a)  STFT ')

In [ ]:
plot.umap_on_condition(df_e_wise,y,'STFT ',df_type=False)

In [28]:
dff=df[['RR', 'DET', 'L', 'Lmax', 'DIV',
        'Lentr', 'DET_RR', 'LAM', 'V',
        'Vmax', 'Ventr', 'LAM_DET', 'W',
        'Wmax', 'Wentr', 'TT']]

features=['RR', 'DET', 'L', 'Lmax', 'DIV',
          'Lentr', 'DET_RR', 'LAM', 'V',
          'Vmax', 'Ventr', 'LAM_DET', 'W',
          'Wmax', 'Wentr', 'TT']


In [ ]:
df

In [ ]:
import rcr
import numpy as np
import pandas as pd
df_ = df
stats_data_cleaned=df_[['RR', 'DET', 'L', 'Lmax', 'DIV',
                        'Lentr', 'DET_RR', 'LAM', 'V',
                        'Vmax', 'Ventr', 'LAM_DET', 'W',
                        'Wmax', 'Wentr', 'TT']]
stats_data_cleaned[:]=np.nan
r = rcr.RCR(rcr.SS_MEDIAN_DL)

for f in features:

    r.performBulkRejection(df_[f])
    cleaned_data_indices = r.result.indices
    stats_data_cleaned[f][cleaned_data_indices]=dff[f][cleaned_data_indices]

In [79]:
# df = features_space.load_features_csv_concat('/Users/luki/DataspellProjects/BrainPulse_Project/Results/erp_stft')
#
# df_stats_data_cleaned = pd.DataFrame(df, columns=['RR', 'DET', 'L', 'Lmax', 'DIV',
#                                                   'Lentr', 'DET_RR', 'LAM', 'V',
#                                                   'Vmax', 'Ventr', 'LAM_DET', 'W',
#                                                   'Wmax', 'Wentr', 'TT'])
# df[['RR', 'DET', 'L', 'Lmax', 'DIV',
#     'Lentr', 'DET_RR', 'LAM', 'V',
#     'Vmax', 'Ventr', 'LAM_DET', 'W',
#     'Wmax', 'Wentr', 'TT']] = df_stats_data_cleaned
#
# cols_to_norm = ['RR', 'DET', 'L', 'Lmax', 'DIV',
#                 'Lentr', 'DET_RR', 'LAM', 'V',
#                 'Vmax', 'Ventr', 'LAM_DET', 'W',
#                 'Wmax', 'Wentr', 'TT']
# df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()), axis=0)

In [ ]:
# import matplotlib.pyplot as plt
# plt.style.use('dark_background')
# df = df.iloc[: , 1:]
# fig, axs = plt.subplots(nrows=len(cols_to_norm),ncols=1,figsize=(10,30),dpi=240)
# for i,n in enumerate(axs):
#     df[cols_to_norm[i]].plot(ax = n,title=cols_to_norm[i])
# # plt.tight_layout()

In [ ]:
plot.features_hists(df=df,features_list= ['TT', 'RR', 'DET', 'LAM', 'L', 'Lentr'], condition='Task')

In [ ]:
df

In [ ]:
plot.features_per_subjects_violin(df=df,features_list=['Lentr', 'TT', 'L'], condition='Task')


In [ ]:
# Subject_id = 'S'+ str(f'{sub_id:03}')
# df_Subject = df.groupby('Subject')
# S1 = df_Subject.get_group('S001')
# Task = S1.groupby('Task')
# S1_A = Task.get_group('A')
# S1_AV = Task.get_group('AV')
# electrode_A = S1_A.groupby('Electrode')
# electrode_AV = S1_AV.groupby('Electrode')
#
# features_A = electrode_A['Lentr', 'TT', 'L', 'RR', 'LAM', 'DET', 'V','Vmax', 'Ventr', 'W', 'Wentr'].mean().apply(lambda x:x)
# names_cols_A = pd.DataFrame(np.array([['S001']*16,['A']*16]).T,columns=['Subject', 'Task'])
#
# features_AV = electrode_AV['Lentr', 'TT', 'L', 'RR', 'LAM', 'DET', 'V','Vmax', 'Ventr', 'W', 'Wentr'].mean().apply(lambda x:x)
# names_cols_AV = pd.DataFrame(np.array([['S001']*16,['AV']*16]).T,columns=['Subject', 'Task'])
#
# newA = pd.concat([names_cols_A,features_A.reset_index()],axis=1)
# newAV = pd.concat([names_cols_AV,features_AV.reset_index()],axis=1)
# subjectMean = pd.concat([newA,newAV]).reset_index(drop=True)
# subjectMean